# IMPORT LIBRARIES

In [1]:
# Import Spark libraries
from pyspark.sql import SparkSession
# For Schema 
from pyspark.sql.types import *
# For Column
from pyspark.sql.functions import col
# For Timestamp
from pyspark.sql.functions import current_timestamp
# Creates a Column Object From a Literal Value 
from pyspark.sql.functions import lit
# To Timestamp & Concatenate
from pyspark.sql.functions import to_timestamp, concat 
# Distinct Counts
from pyspark.sql.functions import countDistinct 

# INITIATE SPARK SESSION

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DeltaLake") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/oasis/sources/spark-3.2.0/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/oasis/.ivy2/cache
The jars for the packages stored in: /Users/oasis/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb09311d-2676-400b-b169-f5322567b3d4;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 192ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evi

# INITIALIZE VARIABLES

In [3]:
processed_layer = "s3a://oasiscorp-curated/formula-oasis"

# READ THE DATA

In [4]:
# Read the "races" table
races_df = spark.read.parquet(f"{processed_layer}/races") \
    .withColumnRenamed("name", "race_name")


# Read the "circuits" table
circuits_df = spark.read.parquet(f"{processed_layer}/circuits") \
                            .withColumnRenamed("name", "circuit_name")

# Read the "drivers" table
drivers_df = spark.read.parquet(f"{processed_layer}/drivers")

# Read the "constructors" table
constructors_df = spark.read.parquet(f"{processed_layer}/constructors")

# Read the "results" table
results_df = spark.read.parquet(f"{processed_layer}/results")

23/10/10 15:11:24 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
races_df.show(5)

+-------+---------+----------+-----+--------------------+--------------------+-------------------+
|race_id|race_year|circuit_id|round|           race_name|       ingested_date|     race_timestamp|
+-------+---------+----------+-----+--------------------+--------------------+-------------------+
|      1|     2009|         1|    1|Australian Grand ...|2023-10-10 15:08:...|2009-03-29 06:00:00|
|      2|     2009|         2|    2|Malaysian Grand Prix|2023-10-10 15:08:...|2009-04-05 09:00:00|
|      3|     2009|        17|    3|  Chinese Grand Prix|2023-10-10 15:08:...|2009-04-19 07:00:00|
|      4|     2009|         3|    4|  Bahrain Grand Prix|2023-10-10 15:08:...|2009-04-26 12:00:00|
|      5|     2009|         4|    5|  Spanish Grand Prix|2023-10-10 15:08:...|2009-05-10 12:00:00|
+-------+---------+----------+-----+--------------------+--------------------+-------------------+
only showing top 5 rows



# REGISTER DATA AS TEMPORARY TABLE

In [6]:
# register table as temp view

circuits_df.createOrReplaceTempView("circuit")
drivers_df.createOrReplaceTempView("driver")
constructors_df.createOrReplaceTempView("constructor")
results_df.createOrReplaceTempView("result")
races_df.createOrReplaceTempView("races")

# PERFORM SQL JOIN